In [1]:
import kwant 
import matplotlib.pyplot as plt
import numpy as np
from dataclasses import dataclass, asdict
from kwant.continuum import discretize  
sx = np.array([[0 , 1] , [1 , 0]])
sy = np.array([[0 , -1j] , [1j , 0]])
sz = np.array([[1 , 0] , [0 , -1]])
id = np.array([[1 , 0] , [0 , 1]])

%matplotlib widget

/Users/krishnanganesh/anaconda3/lib/python3.8/site-packages/kwant/solvers/default.py:16: RuntimeWarning: MUMPS is not available, SciPy built-in solver will be used as a fallback. Performance can be very poor in this case.
  warnings.warn("MUMPS is not available, "


In [17]:
from scipy import constants
from sympy import exp
from josephson_current_tools import constants
@dataclass
class parameters:
    a : float # Lattice constant
    L_x : float # Dimension of planar system in x direction
    L_y : float # Dimension of planar system in y direction
    W : float # Width of the normal region in y direction
    m_eff : float # Effective mass
    mu : float # Chemical potential  
    alpha : float # Rashba coupling
    E_z : float # In plane Zeeman
    Delta : float # Superconducting gap
    phase : float # Phase bias of Josephson junction
    flux : float
    k : float # Boltzmann constant
    T : float # Temperature
    current_unit: float # Current unit

def make_junction_with_vortices(params):
    # Make a Rashba josephson junction with Josephson vortices.
    
    # Scaled Parameters:
    lx = int(params.L_x // params.a)
    ly = int(params.L_y // params.a)
    w = int(params.W // params.a)
    halfly = ly // 2
    halfw = w // 2

    lat = kwant.lattice.square(a = params.a, norbs = 4)
    syst = kwant.Builder(particle_hole = np.kron(sy , sy)) 
    
    m = halfly
    # Phase profile due to magnetic field:
    def phases(i , j):
        if (m + halfw)<= j < ly and 0<= i < lx:
            return (2*np.pi)*(params.flux)*(1/lx)*i + (2*np.pi*params.phase)
        else:
            return 0

    t = 0.5*(1 / (params.m_eff *(params.a**2)))
    alph = params.alpha / params.a
    # Defining lattice hoppings:
    for i in range(lx):
        for j in range(ly):
            if 0 <= j <= (m-halfw):
                syst[lat(i , j)] = 4*t*np.kron(sz , id) - params.mu*np.kron(sz , id) + 0.5*(params.m_eff*(params.alpha**2))*np.kron(sz , id) + params.Delta*np.kron(sx , id)

            if (m-halfw)< j < (m+halfw):
                syst[lat(i , j)] = 4*t*np.kron(sz , id) - params.mu*np.kron(sz , id) + 0.5*(params.m_eff*(params.alpha**2))*np.kron(sz , id) + params.E_z*np.kron(id , sx)

            if (m+halfw)<= j < ly:
                syst[lat(i , j)] = (4*t*np.kron(sz , id) - params.mu*np.kron(sz , id) + 0.5*(params.m_eff*(params.alpha**2))*np.kron(sz , id) + 0.5*params.Delta*np.exp(1j*phases(i,j))*np.kron(sx + 1j*sy , id)
                                    + 0.5*params.Delta*np.exp(-1j*phases(i,j))*np.kron(sx - 1j*sy , id))


            if j > 0:
                    syst[lat(i , j) , lat(i , j-1)] = -1j*0.5*(alph)*np.kron(sz , sx) - (t)*np.kron(sz , id)

            if i > 0:
                syst[lat(i , j) , lat(i-1 , j)] = +1j*0.5*(alph)*np.kron(sz , sy) - (t)*np.kron(sz , id)
    
    syst = syst.finalized()

    return syst

In [2]:
params = parameters(a = 0.5 , L_x = 100, L_y = 21 , W  = 2 , m_eff = 0.5 , mu = 2.0 , alpha = 1 , E_z= 0.68 , Delta = 4.0 , phase = 0.76 , flux = 1, k = constants.k , T = 0.05 , current_unit = constants.current_unit)
syst = make_junction_with_vortices(params)
import scipy.sparse.linalg as sla
ham = syst.hamiltonian_submatrix(params = asdict(params) , sparse = True)
# These are various solving and plotting functions:
def solve_me(ham, number):
    eigenvalues , eigenvectors = sla.eigsh(ham , k = number , sigma = 0.0 , which = 'LM')

    idx = np.argsort(np.abs(eigenvalues))
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[: , idx]

    return eigenvalues , eigenvectors

evals , evecs = solve_me(ham , 40)

plt.figure()
plt.scatter(np.arange(len(evals)),evals[np.argsort(evals)])
print(evals)

NameError: name 'parameters' is not defined

In [1]:
def see_the_funk(k , ev , params):
    psi = ev[: , k]
    innerprod = np.conjugate(psi) * psi
    prob_density = np.array([sum(innerprod[i : i + 4]) for i in range(0 , len(innerprod), 4)])
    wavefunc = np.reshape(prob_density , ((int(params.L_x/params.a)) , int(params.L_y/params.a)))
    return np.real(wavefunc.T)

wf = see_the_funk(0 , evecs , params)
fig , ax = plt.subplots(nrows = 1 , ncols = 2)
fig.set_figwidth(10)
fig.set_figheight(5)
ax[0].imshow(wf , cmap = 'inferno')
ax[1].plot(wf[wf.shape[0]//2 , :])
plt.tight_layout()

NameError: name 'evecs' is not defined

In [202]:
from scipy.special import erf 

print(erf(10 + 10j))



(0.9616493742724749-0.010987684608193982j)
